In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!zip -r /content/my_drive.zip /content/drive/MyDrive -x "/content/drive/MyDrive/root_cifar10/*" "/content/drive/MyDrive/root_cifar10_test/*" "/content/drive/MyDrive/root_mnist/*" "/content/drive/MyDrive/root_mnist_test/*" "/content/drive/MyDrive/test_results/*" "/content/drive/MyDrive/train_results/*"

Scanning files ................
  adding: content/drive/MyDrive/ (stored 0%)
  adding: content/drive/MyDrive/Cold-Diffusion-Models/ (stored 0%)
  adding: content/drive/MyDrive/Cold-Diffusion-Models/README.md (deflated 80%)
  adding: content/drive/MyDrive/Cold-Diffusion-Models/all_transform_cover.png (deflated 4%)
  adding: content/drive/MyDrive/Cold-Diffusion-Models/snowification/ (stored 0%)
  adding: content/drive/MyDrive/Cold-Diffusion-Models/snowification/testing_script.sh (deflated 60%)
  adding: content/drive/MyDrive/Cold-Diffusion-Models/snowification/setup.py (deflated 48%)
  adding: content/drive/MyDrive/Cold-Diffusion-Models/snowification/Fid/ (stored 0%)
  adding: content/drive/MyDrive/Cold-Diffusion-Models/snowification/Fid/inception.py (deflated 76%)
  adding: content/drive/MyDrive/Cold-Diffusion-Models/snowification/Fid/fid_score.py (deflated 72%)
  adding: content/drive/MyDrive/Cold-Diffusion-Models/snowification/Fid/__init__.py (deflated 21%)
  adding: content/drive/MyD

In [ ]:
# necessary packages

!pip install einops
!pip install torchgeometry
!pip install kornia
!pip install pytorch_msssim


%load_ext autoreload
%autoreload 2

**LOAD DATASET (RUN IF DATA NOT AVAILABLE)**

In [ ]:
# Deactivate comment columns for loading another dataset
# Also control root folder to be in drive path
!python3 /content/drive/MyDrive/Cold-Diffusion-Models/create_data.py

## Snowification


Train


In [ ]:
!python3 '/content/drive/MyDrive/Cold-Diffusion-Models/snowification/train.py' --model 'UnetResNet' --dataset 'cifar10' --save_and_sample_every 99 --time_steps 200 --train_steps 100 --sampling_routine x0_step_down --snow_level 3 --random_snow --save_folder '/content/drive/MyDrive/train_results/snow_cifar10_train_results_04_12' --dataset_folder '/content/drive/MyDrive/root_cifar10' --exp_name 'trial' --forward_process_type 'Snow'

Test

In [ ]:
!python3 '/content/drive/MyDrive/Cold-Diffusion-Models/snowification/test.py' --model 'UnetResNet' --dataset 'cifar10' --time_steps 200 --sampling_routine 'x0_step_down' --test_type 'test_data' --save_folder_train '/content/drive/MyDrive/train_results/snow_cifar10_train_results_04_12' --save_folder_test '/content/drive/MyDrive/test_results/snow_cifar10_test_results_04_12' --dataset_folder '/content/drive/MyDrive/custom_cifar10_test' --exp_name 'trial' --forward_process_type 'Snow'

## Decolorization

Train

In [ ]:
!python3 '/content/drive/MyDrive/Cold-Diffusion-Models/decolor-diffusion/train.py' --train_steps 100  --forward_process_type 'Decolorization' --decolor_total_remove --decolor_routine 'Linear' --save_folder '/content/drive/MyDrive/train_results/decolor_cifar10_train_results' --dataset_folder '/content/drive/MyDrive/root_cifar10' --exp_name 'trial'

Test

In [ ]:
!python3 '/content/drive/MyDrive/Cold-Diffusion-Models/decolor-diffusion/test.py' --train_steps 1000  --forward_process_type 'Decolorization' --decolor_total_remove --decolor_routine 'Linear' --test_type 'test_data' --save_folder_train '/content/drive/MyDrive/train_results/decolor_train_results' --save_folder_test '/content/drive/MyDrive/test_results/decolor_test_results' --dataset_folder '/content/drive/MyDrive/root_cifar10' --exp_name 'trial' --order_seed 1

## Resolution

Train

In [ ]:
# mnist train with resolution
# time steps should be less than image size
!python3 '/content/drive/MyDrive/Cold-Diffusion-Models/resolution-diffusion-pytorch/mnist_train.py' --time_steps 27 --train_steps 41 --resolution_routine 'Incremental' --save_folder '/content/drive/MyDrive/train_results/resolution_mnist_train_results' --data_path '/content/drive/MyDrive/root_mnist'

Test

In [ ]:
# mnist test with resolution
!python3 '/content/drive/MyDrive/Cold-Diffusion-Models/resolution-diffusion-pytorch/mnist_test.py' --data_path '/content/drive/MyDrive/root_mnist_test' --time_steps 27 --train_routine 'Final' --sampling_routine 'x0_step_down' --resolution_routine 'Incremental' --load_path '/content/drive/MyDrive/train_results/resolution_mnist_train_results/model.pt' --save_folder  '/content/drive/MyDrive/test_results/resolution_mnist_test_results' --test_type 'test_data'

## Adversarial Attack

In [ ]:
import os
import re
import sys
import tensorflow as tf
import numpy as np
from PIL import Image

sys.path.append("/content/drive/MyDrive/python_files")
from adversarial_attack import AdversarialAttack

adversarial_attack = AdversarialAttack()
adversarial_attack.load_model()
epsilon = 0.075

label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

image_directory = "/content/drive/MyDrive/custom_cifar10_test"

def get_label_index(filename):
    for label in label_names:
        if re.match(f"{label}[^a-zA-Z]*", filename):
            return label_names.index(label)
    return None

for filename in os.listdir(image_directory):
    if filename.endswith(".png"):
        image_path = os.path.join(image_directory, filename)
        img = Image.open(image_path).convert('RGB')

        # Resize the image using LANCZOS
        img = img.resize((32, 32), Image.LANCZOS)

        label = get_label_index(filename)
        if not label:
          print("no label")
          continue

        one_hot_label = tf.keras.utils.to_categorical(label, 10)

        # Convert the PIL Image to a NumPy array, normalize it and ensure correct data type
        img_np = np.array(img) / 255.0
        img_np = img_np.astype(np.float32)

        try:
            perturbations = adversarial_attack.generate_adversary(img_np, one_hot_label).numpy()

            perturbations = np.squeeze(perturbations, axis=0).transpose(1, 2, 0)

            adversarial = img_np + (perturbations * epsilon)
            adversarial = np.clip(adversarial, 0, 1)
            adversarial_image = Image.fromarray((adversarial * 255).astype(np.uint8))
            adversarial_image.save(os.path.join(image_directory, 'adv_' + filename))
        except Exception as e:
            print(f"Error processing {filename}: {e}")



## Load ResNet for Cifar-10

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch import nn, optim

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to the size that ResNet expects
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet mean and std
])


In [ ]:
trainset = ImageFolder(root='/content/drive/MyDrive/root_cifar10/', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = ImageFolder(root='/content/drive/MyDrive/root_cifar10_test/', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
resnet = models.resnet18(pretrained=False)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 10)  # Adjusting the final layer for CIFAR-10

# Load the state dict saved earlier
state_dict = torch.load('/content/drive/MyDrive/cifar10_resnet18.pth',map_location=torch.device('cpu'))
resnet.load_state_dict(state_dict)

# Freeze all the layers in the network except final
for param in resnet.fc.parameters():
  param.requires_grad = True

# Move the model to the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet = resnet.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_epochs = 3

# Train the model
for epoch in range(num_epochs):
    resnet.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = resnet(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss/len(trainloader)}')

print('Finished Training')


In [ ]:
torch.save(resnet.state_dict(), '/content/drive/MyDrive/cifar10_resnet18.pth')

In [ ]:
resnet.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        print(inputs.shape)
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total}%')


In [ ]:
from PIL import Image
import torch.nn.functional as F

cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


def predict(image_path, model, device):
    # Load and transform the image
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)  # Apply the same transformations and add batch dimension

    # Put the model in evaluation mode and make a prediction
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        probabilities = F.softmax(outputs, dim=1)
        class_probabilities, predicted = torch.max(probabilities, 1)
        predicted_class = cifar10_classes[predicted.item()]
        predicted_probability = class_probabilities.item()

    return predicted_class, predicted_probability


In [ ]:
print("Original:    ", predict("/content/drive/MyDrive/cat2.png", resnet, "cuda:0" if torch.cuda.is_available() else "cpu"))
print("Attacked:    ", predict("/content/drive/MyDrive/custom_cifar10_test/adv_cat2.png", resnet, "cuda:0" if torch.cuda.is_available() else "cpu"))
print("Snowified:   ", predict("/content/drive/MyDrive/test_results/snow_cifar10_test_results_04_12/trial/test_data/no_gif/sample-0-10-xt.png", resnet, "cuda:0" if torch.cuda.is_available() else "cpu"))
print("Cleared snow:", predict("/content/drive/MyDrive/test_results/snow_cifar10_test_results_04_12/trial/test_data/no_gif/sample-199-10-xt.png", resnet, "cuda:0" if torch.cuda.is_available() else "cpu"))


Original:     ('cat', 0.9999656677246094)
Attacked:     ('cat', 0.5933854579925537)
Snowified:    ('cat', 0.6036026477813721)
Cleared snow: ('cat', 0.5864997506141663)


##DENEME

In [ ]:
import tensorflow as tf
import numpy as np
import torch
from torchsummary import summary


# Define the adversarial attack function
def generate_adversarial_example(image, label, model, epsilon=0.01):
    """
    with tf.GradientTape() as tape:
        image.requires_grad = True
        prediction = model(image)
        loss = criterion(torch.tensor(prediction), torch.tensor(label, dtype=torch.long))

    gradient_image, gradient_label = torch.autograd.grad(loss, [image, label])

    perturbation = epsilon * tf.sign(gradient_image)

    adv_example = image + perturbation
    adv_example = tf.clip_by_value(adv_example, 0, 255)

    return adv_example.numpy()
    """

    with tf.GradientTape() as tape:
      prediction = model(image)
      loss = tf.keras.losses.MSE(label, prediction)
    gradient = tape.gradient(loss, image)
    sign_grad = tf.sign(gradient)

# Apply adversarial attack to the entire dataset
epsilon = 0.01  # Adjust as needed
adversarial_images = []

for inputs, labels in testloader:

    device = "cpu"
    inputs, labels = inputs.to(device), labels.to(device)

    # Convert NumPy arrays to TensorFlow tensors
    inputs_torch = torch.tensor(inputs, dtype=torch.float32)
    #labels_torch = torch.tensor(labels, dtype=torch.int64)

    # Generate adversarial example using TensorFlow code
    adv_image = generate_adversarial_example(inputs_torch, labels, resnet.to(device), epsilon)

    # Convert TensorFlow tensor to NumPy array for further processing if needed
    adv_image_np = adv_image.numpy()

    # Now you can use adv_image_np as needed


# TRY TO TRAIN WITH ADDITIONAL ADVERSIAL INPUTS


In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/python_files")
from models import Models
from adversial_attack import AdversarialAttack
sys.path.append("/content/drive/MyDrive/Cold-Diffusion-Models/snowification/diffusion/model")
from unet_resnet import UnetResNetBlock

%load_ext autoreload
%autoreload 2


In [ ]:
resnet = UnetResNetBlock(resolution=32,
                in_channels=3,
                out_ch=3,
                ch=128,
                ch_mult=(1,2,2,2),
                num_res_blocks=2,
                attn_resolutions=(16,),
                with_time_emb=False,
                dropout=0.1
            )

def load(load_path):
        print("Loading : ", load_path)
        data = torch.load(load_path,map_location=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
        resnet.load_state_dict(data['model'], strict=False)

load('/content/drive/MyDrive/train_results/trial/model.pt')

Loading :  /content/drive/MyDrive/train_results/trial/model.pt


In [ ]:
adv_attack = AdversarialAttack()
adv_attack.load_model()
#adv_in,adv_out = next(adv_attack.adversary_generator(10000))

model loaded: /content/drive/MyDrive/cifar10_resnet18.pth


In [ ]:
adv_in,adv_out = next(adv_attack.adversary_generator(1000))

0 images generated


In [ ]:
adv_in.shape

(1000, 32, 32, 3)

In [ ]:
adv_out.shape

(1000, 10)

In [ ]:
from torch.optim import Adam
try:
    from apex import amp
    APEX_AVAILABLE = True
except:
    APEX_AVAILABLE = False
from functools import partial

def loss_backwards(fp16, loss, optimizer, **kwargs):
    if fp16:
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward(**kwargs)
    else:
        loss.backward(**kwargs)

backwards = partial(loss_backwards, False)
step = 0
opt = Adam(resnet.parameters())
criterion = torch.nn.CrossEntropyLoss()

while step < 10:
  for i in range(2):
      adv_in,adv_out = next(adv_attack.adversary_generator(32))
      original_shape = adv_in.shape
      adv_in = adv_in.reshape((original_shape[0], original_shape[-1], *original_shape[1:-1]))
      adv_in = torch.tensor(adv_in, dtype=torch.float32)
      adv_out = torch.tensor(adv_out, dtype=torch.long)
      logits = resnet(adv_in)

      loss = criterion(logits, adv_out)
      print(f'{step}: {loss.item()}')
      backwards(loss / 2, opt)
  opt.step()
  opt.zero_grad()

  step += 1

In [ ]:
"""
import torch
import torchvision
import torchvision.transforms as transforms

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
])

# Create CIFAR-10 train and test datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
"""
# Check the shape of the dataset
print(type(train_dataset))
print(train_dataset.data.shape)
print(len(train_dataset.targets))


<class 'torchvision.datasets.cifar.CIFAR10'>
(50000, 32, 32, 3)
50000
